In [1]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.7: Fast Qwen3 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.8.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [2]:
new_chat_template = \
    "{% for message in messages %}"\
        "{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>' + '\\n'}}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '<|im_start|>assistant\\n' }}"\
    "{% endif %}"
original_chat_template = None
if original_chat_template is None:
    original_chat_template = tokenizer.chat_template
tokenizer.chat_template = new_chat_template

tokenizer.apply_chat_template([
        {
            "role": "user", 
            "content": "Write a satirical headline in the style of Onion News. Only write the headline."
        },
        {
            "role": "assistant",
            "content": "Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage"
        },
    ], tokenize=False)

'<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n'

In [3]:
import random
from datasets import load_dataset, load_from_disk


def get_as_messages(data):
    headline = get_headline(data['text'])
    messages = get_prompt()
    messages.append(format_reply(headline))
    return {'messages': messages}

SEP = '#~#'
def get_headline(text):
    return text.split(SEP)[0].strip()

def get_prompt():
    return [
        {
            "role": "user", 
            "content": get_prompt_no_topic(),
        }
    ]

def get_prompt_no_topic():
    return random.choice([
        'Write a satirical headline in the style of Onion News.',
        'Generate an satirical news headline.',
        "Give me a headline that sounds like it's from the Onion.",
        'Come up with a funny, fake news headline.',
        'Write a headline that is both absurd and oddly believable.',
        'Please write a headline that would feel at home in the Onion.',
        'Can you write a satirical headline for me?',
    ])

def format_reply(headline):
    return {
        "role": "assistant",
        "content": headline,
    }

def apply_chat_template(data):
    messages = data['messages']
    chat = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=False)
    return {'text': chat}
    

filepath = 'Onion-News-Tokens'
try:
    ds = load_from_disk(filepath)
except:
    ds = load_dataset("Biddls/Onion_News")
    ds = ds.map(get_as_messages, remove_columns=['text'])
    ds = ds.map(apply_chat_template, remove_columns=['messages'])
    ds.save_to_disk(filepath)
ds['train'][:5]

Map:   0%|          | 0/33880 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/33880 [00:00<?, ? examples/s]

{'text': ['<|im_start|>user\nWrite a headline that is both absurd and oddly believable.<|im_end|>\n<|im_start|>assistant\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n',
  '<|im_start|>user\nPlease write a headline that would feel at home in the Onion.<|im_end|>\n<|im_start|>assistant\nU.S. Officials Call For Correct Amount Of Violence<|im_end|>\n',
  "<|im_start|>user\nGive me a headline that sounds like it's from the Onion.<|im_end|>\n<|im_start|>assistant\nKamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime<|im_end|>\n",
  '<|im_start|>user\nCome up with a funny, fake news headline.<|im_end|>\n<|im_start|>assistant\n25 Arrested In Fake Nursing School Diploma Scheme<|im_end|>\n',
  '<|im_start|>user\nWrite a satirical headline in the style of Onion News.<|im_end|>\n<|im_start|>assistant\nWorld’s Oldest American Dies At 72<|im_end|>\n']}

In [4]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds['train'],
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        
        output_dir='./qwen3_lora_finetuned_v3.1',
        save_steps=500,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/33880 [00:00<?, ? examples/s]

In [5]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=12):   0%|          | 0/33880 [00:00<?, ? examples/s]

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,880 | Num Epochs = 1 | Total steps = 4,235
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 87,293,952 of 8,278,029,312 (1.05% trained)


Step,Training Loss
10,8.083400
20,5.461100
30,4.215400
40,3.935500
50,3.732200
60,3.784300
70,3.742400
80,3.697300
90,3.616000
100,3.598500


Unsloth: Will smartly offload gradients to save VRAM!
